# TODO

# Imports

In [1]:
# ============================================================================
#   TO-DO
# ----------------------------------------------------------------------------
# - Seed
# - Make videos

#Secondaries
# - Maybe gen model predictive videos?

# ============================================================================
#   IMPORTS
# ----------------------------------------------------------------------------

import shutil
import zipfile
import datetime

import subprocess

import gc
import os
import pathlib
import random 
import torch

import ruamel.yaml as yaml

from pathlib import Path
from tensordict import TensorDict
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.storages import LazyTensorStorage, LazyMemmapStorage
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.transforms import ToTensorImage
from types import SimpleNamespace

import dreamer
import tools

from torchrl.envs import (
    Compose,
    InitTracker,
    ObservationNorm,
    PermuteTransform,
    RewardScaling,
    StepCounter,
    TransformedEnv,
)


c:\Users\as1748\rayenv\lib\site-packages\torchrl\data\replay_buffers\samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)
c:\Users\as1748\rayenv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\as1748\rayenv\lib\site-packages\tensorflow\python\framework\dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


# Functions

In [ ]:
def load_data(replay_buffer, data_path, batch_length=-1, randomize=True, indices_to_exclude=None):
    """
    Load data from a memmapped TensorDict and populate the given replay buffer.

    Args:
        replay_buffer (ReplayBuffer): The buffer to populate with loaded samples.
        data_path (str or Path): Path to the memmapped TensorDict.
        batch_length (int): Number of trajectories to load. If -1, load all.
        randomize (bool): Whether to shuffle the loaded indices.
        indices_to_exclude (int or list[int], optional): Indices to skip during loading.

    Raises:
        ValueError: If indices_to_exclude is not None, int, or list of ints.
    """
    
    # Load the TensorDict from the memmap
    loaded = TensorDict.load_memmap(data_path)

    # Ensure indices_to_exclude is a list
    if indices_to_exclude is None:
        indices_to_exclude = []
    elif isinstance(indices_to_exclude, int):
        indices_to_exclude = [indices_to_exclude]
    elif not isinstance(indices_to_exclude, list):
        raise ValueError("indices_to_exclude must be an int or a list of ints")
    
    if batch_length > 0:
        # Create a list of indices to iterate over, excluding specified indices
        indices = [i for i in range(batch_length) if i not in indices_to_exclude]
    else:
        # Create a list of indices to iterate over, excluding specified indices
        indices = [i for i in range(len(loaded)) if i not in indices_to_exclude]

    # Randomize the indices if requested
    if randomize:
        random.shuffle(indices)

    # Extend the replay buffer with data at each index
    for i in indices:
        replay_buffer.extend(loaded[i:i+1])

    del loaded

    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()

def get_git_commit():
    try:
        commit = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode().strip()
        dirty = subprocess.check_output(["git", "status", "--porcelain"]).decode().strip()
        return commit + ("-dirty" if dirty else "")
    except Exception as e:
        return f"Git info unavailable: {e}"
    
def make_config(path="configs.yaml"):
    """
    Load and parse the configuration from a YAML file into a SimpleNamespace.

    Args:
        path (str): Path to the YAML configuration file.

    Returns:
        SimpleNamespace: Config object with attribute access to nested values.
    """
    def recursive_update(base, update):
        """
        Recursively merges `update` into `base`, modifying `base` in-place.
        """
        for key, value in update.items():
            if isinstance(value, dict) and key in base:
                recursive_update(base[key], value)
            else:
                base[key] = value

    def dict_to_namespace(d):
        """
        Converts a nested dictionary into a nested SimpleNamespace.
        """
        return SimpleNamespace(**{k: v for k, v in d.items()})

    yaml_parser = yaml.YAML(typ='safe', pure=True)
    config_yaml = yaml_parser.load((pathlib.Path(path)).read_text())
    config = {}
    recursive_update(config, config_yaml['defaults'])
    return dict_to_namespace(config)


# Setup

In [3]:
if torch.cuda.is_available():
    device = torch.device(0)
    print('CUDA initialised')
else:
    print("WARNING - CUDA not available - no hardware acceleration being used")

config = make_config()
tools.set_seed_everywhere(config.seed)

experiment_name = config.exp_name
exp_date = config.exp_date
path_root = config.path_root
data_path_preloaded = config.data_path_preloaded

results_path = f"{path_root}\\{experiment_name}\\results"
save_path = f"{path_root}\\{experiment_name}\\saved_models"
logdir = Path(f"{path_root}\\{experiment_name}\\logs")
data_path = f"{path_root}\\{experiment_name}\\dataset"

for p in [results_path, f"{results_path}\\{exp_date}", save_path, logdir, data_path]:
    os.makedirs(p, exist_ok=True)

# Create experiment folder
root_dir = Path(os.getcwd())
experiments_dir = root_dir / "experiments"
experiment_folder = experiments_dir / config.exp_date

# Delete if exists, then recreate
if experiment_folder.exists():
    shutil.rmtree(experiment_folder)
experiment_folder.mkdir(parents=True, exist_ok=True)


CUDA initialised


In [4]:
from importlib import reload
import exploration as expl

reload(expl)
reload(dreamer)

<module 'dreamer' from 'c:\\Users\\as1748\\projects\\dreamer_w_ant_cur\\dream3_solution_1_clean_w_rand_start_hard\\dreamer.py'>

In [5]:

# ============================================================================
#   GLOBALS
# ----------------------------------------------------------------------------

# Initialize the replay buffer
replay_buffer = ReplayBuffer(
    storage=LazyMemmapStorage(
        max_size=int(config.dataset_size / config.time_limit),
        scratch_dir=data_path + '//' + exp_date
    ),
    prefetch=1,
    batch_size=torch.Size([256, config.time_limit]),
)

# Initialize temp buffer (if needed for short-term storage)
temp_buffer = ReplayBuffer(
    storage=LazyTensorStorage(max_size=config.time_limit),
)

logger = tools.Logger(logdir/exp_date, 0)

#Make env
env = TransformedEnv(
    GymEnv(config.env_name, **config.maze_config),
    Compose(
        ToTensorImage(in_keys=["pixels"], from_int=True), 
        ObservationNorm(in_keys=["pixels"], loc=0, scale=1),
        PermuteTransform((-2, -1, -3), in_keys=["pixels"]),
        InitTracker(),
        RewardScaling(loc=0.0, scale=1),
        StepCounter(),
    ),
)

env_tens = env.reset()

obs_space = {
    'image': env_tens['pixels'][:,:64],
    'map': env_tens['pixels'][:,64:],
}

agent = dreamer.Dreamer(
    obs_space,
    config,
    logger,
).to(config.device)

agent.requires_grad_(requires_grad=False)

torch.cuda.empty_cache()
gc.collect()


c:\Users\as1748\rayenv\lib\site-packages\torchrl\envs\common.py:2989: DeprecationWarning: Your wrapper was not given a device. Currently, this value will default to 'cpu'. From v0.5 it will default to `None`. With a device of None, no device casting is performed and the resulting tensordicts are deviceless. Please set your device accordingly.
  warnings.warn(


Encoder CNN shapes: {'map': (64, 64, 3)}
Encoder MLP shapes: {}
Encoder CNN shapes: {'image': (64, 64, 3)}
Encoder MLP shapes: {}
Decoder CNN shapes: {'image': (64, 64, 3), 'map': (64, 64, 3)}
Decoder MLP shapes: {}
Optimizer model_opt has 23790789 variables.
Optimizer actor_opt has 1053702 variables.
Optimizer value_opt has 1181439 variables.
Biases after reassigning: tensor([0.0075, 0.0525, 0.0525, 0.0578, 0.0578], device='cuda:0')


c:\Users\as1748\projects\dreamer_w_ant_cur\dream3_solution_1_clean_w_rand_start_hard\tools.py:905: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self._scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


144

In [6]:
# ============================================================================
#   MAKE RANDOM DATA - takes up to 60 minutes
# ----------------------------------------------------------------------------
#add this bit on

# ============================================================================
#   LOAD DATA - takes up to 6 minutes
#             - 4, X, 3, 5.5
# ----------------------------------------------------------------------------

load_data(
    replay_buffer=replay_buffer,
    data_path=data_path_preloaded + '//random_data',
)

In [7]:
# ============================================================================
#   REPRODUCABILITY
# ----------------------------------------------------------------------------

# Archive current configs.yaml + all .py files into a zip
archive_path = experiment_folder / f"{config.exp_date}_code_snapshot.zip"
with zipfile.ZipFile(archive_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add configs.yaml
    config_path = root_dir / "configs.yaml"
    zipf.write(config_path, arcname="configs.yaml")
    
    # Add all .py files from the project root
    for py_file in root_dir.glob("*.py"):
        zipf.write(py_file, arcname=py_file.name)
    # Add all .ipynb files from the project root
    for ipynb_file in root_dir.glob("*.ipynb"):
        zipf.write(ipynb_file, arcname=ipynb_file.name)
# Get current datetime
now = datetime.datetime.now()
date_str = now.strftime("%d-%m-%Y")
time_str = now.strftime("%H:%M:%S")

# Write README
with open(experiment_folder / "README.txt", "w") as f:
    f.write(f"Experiment: {experiment_name}\n")
    f.write(f"Name: {exp_date}\n")
    f.write(f"Date: {date_str}\n")
    f.write(f"Start Time: {time_str}\n")
    f.write(f"Git: {get_git_commit()}\n")
    f.write(f"Notes:")

print("Git commit:", get_git_commit())

Git commit: Git info unavailable: Command '['git', 'rev-parse', 'HEAD']' returned non-zero exit status 128.


In [8]:
# from importlib import reload
# import exploration as expl

# # reload(models)
# reload(dreamer)
# reload(expl)
# reload(dreamer)

# agent = dreamer.Dreamer(
#     obs_space,
#     config,
#     logger,
# ).to(config.device)

# agent.requires_grad_(requires_grad=False)

# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
# ============================================================================
#   RUN
# ----------------------------------------------------------------------------

state = None
while agent._step < config.steps:
    logger.write()
    print("Start training.")
    agent.train()
    state = tools.simulate(
        agent,
        env,
        replay_buffer,
        temp_buffer,
        logger,
        steps = agent._config.time_limit,
        state = state,
        config=config)
    items_to_save = {
        "agent_state_dict": agent.state_dict(),
        "optims_state_dict": tools.recursively_collect_optim_state_dict(agent),
    }
    torch.save(items_to_save, logdir / "latest.pt")
    print(agent._step)


[0] 
Start training.


c:\Users\as1748\projects\dreamer_w_ant_cur\dream3_solution_1_clean_w_rand_start_hard\models.py:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k: torch.tensor(v, device='cuda', dtype=torch.float32)
c:\Users\as1748\projects\dreamer_w_ant_cur\dream3_solution_1_clean_w_rand_start_hard\dreamer.py:335: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Episodes done: 1
[1490] buffer_size_ 0.0 / buffer_size__r 0.0 / _plan_prob_metric 0.5 / _plan_prob_metric_std 0.3 / _plan_prob_metric_max 1.0 / _plan_steps_metric 1.0 / _plan_hor_metric 10.1 / _plan_hor_metric_std 2.7 / _plan_hor_metric_max 15.0 / _plan_ent_metric 0.4 / _plan_ent_metric_std 0.2 / _plan_ent_metric_max 1.0 / actual_ent_diff 34.0 / num_replans 108.0 / ent_mean 60.6 / rew_mean -6.4 / final_reward_diff 31.7 / final_delta 0.0 / model_loss 2013.9 / model_grad_norm 2362.5 / image_loss 540.3 / map_loss 1460.3 / reward_loss 1.1 / entropy_loss 1.0 / cont_loss 0.0 / kl_free 1.0 / dyn_scale 0.5 / rep_scale 0.1 / dyn_loss 18.7 / rep_loss 18.7 / kl 18.7 / prior_ent 60.9 / post_ent 40.3 / normed_target_mean 0.5 / normed_target_std 0.3 / normed_target_min -0.2 / normed_target_max 1.1 / EMA_005 -85.8 / EMA_095 -13.5 / value_mean -7.0 / value_std 3.4 / value_min -41.6 / value_max -0.2 / target_mean -48.9 / target_std 22.6 / target_min -103.6 / target_max -6.2 / imag_reward_mean -7.3 / im

In [13]:
import matplotlib.pyplot as plt

In [26]:
import cv2
import numpy as np

# Assume 'frames' is your NumPy array with shape (370, 128, 64, 3)
# And dtype is probably uint8. If it's not, fix it. Don't make me come over there.
frames = temp_buffer['image'].numpy()  # replace with your actual variable
frames = (np.clip(frames, 0, 1) * 255).astype(np.uint8)  # if float images

# Video settings
output_file = results_path + '//matrix_video.mp4'
height, width = frames.shape[1:3]
fps = 10 

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

# Write each frame
for i, frame in enumerate(frames):
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # OpenCV uses BGR, because it's difficult on purpose
    video.write(frame_bgr)

video.release()
print(f"Behold, your video has been forged: {output_file}")

Behold, your video has been forged: C:\\Users\\as1748\\projects\\dreamer_w_ant_cur\\dream3_uq_ent_head_single_disc\plan_tests\results//matrix_video.mp4
